# Predicting Pet Insurance Claims - EDA

In [ ]:
print(str(pets[pets['Species'] == 'Cat']['Breed'].value_counts(sort=True).count()) + " Cat breeds")
print(str(pets[pets['Species'] == 'Dog']['Breed'].value_counts(sort=True).count()) + " Dog breeds") 

Let's go back to the full dataset and review premiums by species.

In [ ]:
# Plot a distribution of Premiums by Species
cat_prem = pets[pets['Species'] == 'Cat']['Premium']
dog_prem = pets[pets['Species'] == 'Dog']['Premium']

plt.hist(dog_prem, bins=50, label='Dogs')
plt.hist(cat_prem, bins=50, label='Cats')
plt.title('Premium Distribution by Species')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Calculate average premium by species
print("The average premium for cats is $" + str(round(pets[pets['Species'] == 'Cat']['Premium'].mean(), 2)))
print("The average premium for dogs is $" + str(round(pets[pets['Species'] == 'Dog']['Premium'].mean(), 2)))

Clearly dog premiums are much higher than cat premiums and it's likely we'll see a similar pattern when we look at claims data later on in EDA. For now, let's plan to drop rows where premiums are in the upper 1% of the range to eliminate the outliers.

In [ ]:
# Calculate the 99th quantile of Premium values
print("99% of premiums are less than $" + str(pets.Premium.quantile(q=0.99)))

Before we move on, let's see how the distribution looks by species. 


In [ ]:
# Plot a distribution of Deductible by Species
cat_prem = pets[pets['Species'] == 'Cat']['Deductible']
dog_prem = pets[pets['Species'] == 'Dog']['Deductible']

plt.hist(dog_prem, bins=20, label='Dogs')
plt.hist(cat_prem, bins=20, label='Cats')
plt.title('Deductible Distribution by Species')
plt.legend(loc='upper right')
plt.show()

In [ ]:
# Review outlier claims
# top_pct = claims.AmountClaimed.quantile(q=0.99)
# print("99% of claims are below $" + str(top_pct))

review of column distributions

In [ ]:
# # Review Distributions of all numeric columns
# merged_pets.hist(figsize=(15,10))
# plt.subplots_adjust(hspace=0.5);

In [ ]:
# Isolate rows with high premiums for additional review
high_prems = pets[pets['Premium'] > 200]

# Review date distribution
plt.figure(figsize=(6, 4))
plt.title('Distribution of High Premiums')
plt.hist(high_prems.Premium, bins = 20)
plt.show()

In [ ]:
# Review the number of pets with high premiums by species
high_prems.Species.value_counts()